<a href="https://colab.research.google.com/github/joseeph/Creative_support_index_gradio_app/blob/main/CSI_survey_UI_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## installation and env setup

In [10]:
pip install gradio

In [11]:
import numpy as np
import gradio as gr

## Mount Google Drive
- link colab to your Google Drive
- set up function to save servey data as csv

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
import pandas as pd


def ensure_directory_exists(directory_path):
    """
    Ensure that a directory exists at the specified 'directory_path'. If the directory does not exist, it will be created.
    """
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

def ensure_csv_file_exists(file_path, header=None):
    """
    Ensure that a CSV file exists at the specified 'file_path'. If the file does not exist, it will be created.
    Optionally, you can provide a 'header' list to be written as the header row in the CSV file.
    """
    if not pd.io.common.file_exists(file_path):
        # If the file doesn't exist, create it and optionally write the header
        df = pd.DataFrame(columns=header) if header else pd.DataFrame()
        df.to_csv(file_path, index=False)

def add_observation_to_csv(file_path, observation):
    """
    Add a new observation to the CSV file at the specified 'file_path'.
    The 'observation' should be a list or a tuple containing the values for each column.
    """
    ensure_csv_file_exists(file_path)  # Make sure the file exists
    df = pd.read_csv(file_path)  # Read existing data

    new_row = pd.Series(observation, index=df.columns)  # Create a new row
    df = df.append(new_row, ignore_index=True)  # Append the new row

    df.to_csv(file_path, index=False)  # Write back to the CSV file

## upload and Save CSI data in Google Drive
- set up directory: change `save_dir='to your own directory in Google Drive'`


In [14]:
def Collect_CSI_Save_CSV(uid, data_rates, data_counts, save_dir='/content/drive/MyDrive/AIGC_Research', csv_name='CSI_data_collection.csv'):
  gr.Info('Submit button clicked!')
  uid = int(uid)

  rates=list(data_rates.values())
  Exploration_rate = rates[0]
  Expressiveness_rate = rates[1]
  Immersion_rate = rates[2]
  EffortResults_rate = rates[3]
  Enjoyment_rate = rates[4]
  Collaboration_rate = rates[5]

  counts=list(data_counts.values())
  Exploration_count = counts[0]
  Expressiveness_count = counts[1]
  Immersion_count = counts[2]
  EffortResults_count = counts[3]
  Enjoyment_count = counts[4]
  Collaboration_count = counts[5]

  CSI_index = (Exploration_rate*Exploration_count
          +Expressiveness_rate*Expressiveness_count
          +Immersion_rate*Immersion_count
          +EffortResults_rate*EffortResults_count
          +Enjoyment_rate*Enjoyment_count
          +Collaboration_rate*Collaboration_count
          )/1.5

  csv_header = ['uid', 'CSI', 'Exploration', 'Expressiveness', 'Immersion', 'EffortResults', 'Enjoyment', 'Collaboration',
                'Exploration_count', 'Expressiveness_count', 'Immersion_count', 'EffortResults_count', 'Enjoyment_count', 'Collaboration_count']


  new_observation = [uid, CSI_index, Exploration_rate, Expressiveness_rate, Immersion_rate, EffortResults_rate, Enjoyment_rate, Collaboration_rate,
                     Exploration_count, Expressiveness_count, Immersion_count, EffortResults_count, Enjoyment_count, Collaboration_count]

  #save to csv
  csv_file_path = os.path.join(save_dir, csv_name)
  ensure_csv_file_exists(csv_file_path, header=csv_header)
  add_observation_to_csv(csv_file_path, new_observation)

  print("uid {} rate CSI:{}".format(uid, CSI_index))

  gr.Info("csv file saved")

## Gradio UI

In [ ]:
# def hello_world(data):
#   gr.Info('Submit button clicked!')
#   count_pair_value(data)
#   # return "Submit Successful!"
#   return count_pair_value(data)

def count_pair_value(state,group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1):
  data = [group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1]
  given_terms = ['Exploration', 'Expressiveness', 'Immersion', 'Result Worth Effort', 'Enjoyment', 'Collaboration']
  # Dictionary to store counts of specific names
  term_counts = {key: 0 for key in given_terms}

  for term in data:
    if term in term_counts:
        term_counts[term] += 1
  # count_state = term_counts.values()
  state = term_counts
  print(term_counts)
  return str(term_counts.values()), state


def count_slider_value(state,slider_1,slider_2,slider_3,slider_4,slider_5,slider_6):
  term_rates = {'Exploration': slider_1,
          'Expressiveness': slider_2,
          'Immersion': slider_3,
          'Result Worth Effort': slider_4,
          'Enjoyment': slider_5,
          'Collaboration': slider_6}
  state = term_rates
  print(term_rates)
  return str(term_rates.values()), state

with gr.Blocks() as demo:
  slider_state = gr.State()
  radio_state = gr.State()
  gr.Markdown("Creative Support index")
  # gr.Markdown(value=count_state)
    # with gr.Tab("Flip Text"):
  # text_input = gr.Textbox(label="User ID")
  uid = gr.Dropdown(choices=list(range(0,19)), label="User ID")
  with gr.Row():
    with gr.Column(scale=3):
      gr.Markdown("Highly Disagree ------------------------------------------------------------------------------------  Highly Agree")
      slider_1 = gr.Slider(0, 10, step=1, label="Result Worth Effort",
                info="What I was able to produce was worth the effort I had to exert to produce it.")
      slider_2 = gr.Slider(0, 10, step=1, label="Expressiveness",
                info="I was able to be very expressive and creative while doing the activity.")
      slider_3 = gr.Slider(0, 10, step=1, label="Exploration",
                info="It was easy for me to explore many different ideas, options, designs, or outcomes.")
      slider_4 = gr.Slider(0, 10, step=1, label="Immersion",
                info="My attention was fully tuned to the activity, and I forgot about the system/tool I was using. ")
      slider_5 = gr.Slider(0, 10, step=1, label="Enjoyment",
                info="I was very engaged in this activity I enjoyed this activity and would do it again.")
      slider_6 = gr.Slider(0, 10, step=1, label="Collaboration",
                info="The system/tool allowed other people to work with me easily.")
    with gr.Column(scale=3):
      gr.Markdown("For each pair below, please select which factor is more important to you when doing the activity:")
      # gr.Markdown(count_state)
      # Exploration
      group_1_1 = gr.Radio(["Exploration", "Collaboration"], show_label=False)
      group_1_2 = gr.Radio(["Exploration", "Enjoyment"], show_label=False)
      group_1_3 = gr.Radio(["Exploration", "Result Worth Effort"], show_label=False)
      group_1_4 = gr.Radio(["Exploration", "Immersion"], show_label=False)
      group_1_5 = gr.Radio(["Exploration", "Expressiveness"], show_label=False)
      # Collaboration
      group_2_1 = gr.Radio(["Collaboration", "Enjoyment"], show_label=False)
      group_2_2 = gr.Radio(["Collaboration", "Result Worth Effort"], show_label=False)
      group_2_3 = gr.Radio(["Collaboration", "Immersion"], show_label=False)
      group_2_4 = gr.Radio(["Collaboration", "Expressiveness"], show_label=False)
      # Expressiveness
      group_3_1 = gr.Radio(["Expressiveness", "Result Worth Effort"], show_label=False)
      group_3_2 = gr.Radio(["Expressiveness", "Immersion"], show_label=False)
      group_3_3 = gr.Radio(["Expressiveness", "Enjoyment"], show_label=False)
      # Enjoyment
      group_4_1 = gr.Radio(["Enjoyment", "Immersion"], show_label=False)
      group_4_2 = gr.Radio(["Enjoyment", "Result Worth Effort"], show_label=False)
      # immersion
      group_5_1 = gr.Radio(["Immersion", "Result Worth Effort"], show_label=False)


  submit_btn = gr.Button(value="Submit")
  md = gr.Markdown()
  mdd = gr.Markdown()
  group_1_1.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_1_2.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_1_3.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_1_4.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_1_5.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])

  group_2_1.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_2_2.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_2_3.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_2_4.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])

  group_3_1.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_3_2.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_3_3.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])

  group_4_1.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  group_4_2.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])

  group_5_1.select(count_pair_value, inputs=[radio_state, group_1_1,group_1_2,group_1_3,group_1_4,group_1_5,group_2_1,group_2_2,group_2_3,group_2_4,group_3_1,group_3_2,group_3_3,group_4_1,group_4_2,group_5_1] ,outputs=[mdd, radio_state])
  slider_1.release(count_slider_value, inputs=[slider_state,slider_1,slider_2,slider_3,slider_4,slider_5,slider_6], outputs=[md, slider_state])
  slider_2.release(count_slider_value, inputs=[slider_state,slider_1,slider_2,slider_3,slider_4,slider_5,slider_6], outputs=[md, slider_state])
  slider_3.release(count_slider_value, inputs=[slider_state,slider_1,slider_2,slider_3,slider_4,slider_5,slider_6], outputs=[md, slider_state])
  slider_4.release(count_slider_value, inputs=[slider_state,slider_1,slider_2,slider_3,slider_4,slider_5,slider_6], outputs=[md, slider_state])
  slider_5.release(count_slider_value, inputs=[slider_state,slider_1,slider_2,slider_3,slider_4,slider_5,slider_6], outputs=[md, slider_state])
  slider_6.release(count_slider_value, inputs=[slider_state,slider_1,slider_2,slider_3,slider_4,slider_5,slider_6], outputs=[md, slider_state])
  submit_btn.click(Collect_CSI_Save_CSV, inputs=[uid, slider_state, radio_state], outputs=[md])
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://def9de0f696ab83cc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


{'Exploration': 7, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 7, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 7, 'Immersion': 7, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 7, 'Immersion': 7, 'EffortResults': 7, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 7, 'Immersion': 7, 'EffortResults': 7, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 7, 'Immersion': 7, 'EffortResults': 7, 'Enjoyment': 7, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 7, 'Immersion': 7, 'EffortResults': 7, 'Enjoyment': 7, 'Collaboration': 7}
{'Exploration': 0, 'Expressiveness': 0, 'Immersion': 0, 'Result Worth Effort': 0, 'Enjoyment': 0, 'Collaboration': 1}
{'Exploration': 0, 'Expressiveness': 0, 'Immersion': 0, 'Result Worth Effort': 1, 'Enjoyment': 0, 

<ipython-input-13-1c8d1ae10514>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)  # Append the new row


uid 2 rate CSI:70.0
{'Exploration': 8, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 0, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 8, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 8, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 9, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 9, 'Immersion': 8, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 7, 'Expressiveness': 9, 'Immersion': 8, 'EffortResults': 5, 'Enjoyme

<ipython-input-13-1c8d1ae10514>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)  # Append the new row


...

uid 11 rate CSI:96.0


<ipython-input-13-1c8d1ae10514>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)  # Append the new row


uid 11 rate CSI:96.0
{'Exploration': 9, 'Expressiveness': 0, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 9, 'Expressiveness': 10, 'Immersion': 0, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 9, 'Expressiveness': 10, 'Immersion': 10, 'EffortResults': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 9, 'Expressiveness': 10, 'Immersion': 10, 'EffortResults': 9, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 9, 'Expressiveness': 10, 'Immersion': 10, 'EffortResults': 9, 'Enjoyment': 10, 'Collaboration': 0}
{'Exploration': 9, 'Expressiveness': 10, 'Immersion': 10, 'EffortResults': 9, 'Enjoyment': 10, 'Collaboration': 9}
{'Exploration': 1, 'Expressiveness': 0, 'Immersion': 0, 'Result Worth Effort': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 2, 'Expressiveness': 0, 'Immersion': 0, 'Result Worth Effort': 0, 'Enjoyment': 0, 'Collaboration': 0}
{'Exploration': 3, 'Expressiveness': 0, 'Immersion': 0, 'Res

<ipython-input-13-1c8d1ae10514>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)  # Append the new row


uid 0 rate CSI:93.33333333333333


## Data Visualization
- histogram